In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

In [37]:
df = pd.read_csv('usina_with_outliers.csv')
df.head()

print("Columns:", df.columns.tolist())
print("\nShape:", df.shape)

print("\nSummary statistics:")
display(df.describe(include="all"))

Columns: ['AT', 'V', 'AP', 'RH', 'PE']

Shape: (9568, 5)

Summary statistics:


,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,19.618518,54.250021,1013.288871,73.308978,454.407820
std,8.256412,13.993655,6.636609,16.094499,18.760047
min,-39.174839,-38.397358,959.607298,-53.091613,327.528030
25%,13.480000,41.670000,1009.077500,63.227500,439.730000
50%,20.320000,52.080000,1012.950000,74.955000,451.620000
75%,25.732500,66.540000,1017.320000,84.882500,468.530000
max,77.344839,155.117358,1064.772702,187.691613,590.091970


Name: Maximiliano González

- Why did you choose this model (Linear vs Ridge vs Lasso)?
    - I chose to work with ridge regression because of its penalty method. Since the independent variables are organized in various orders of magnitude, it is regularize the values to allow for easier comparison of coefficients and reduces the chances of overfitting. Linear regression does not have this penalty function and may ont deal with multicollinearity cases, and lasso regression is used to determine feature importance and rule out potential unimportant features. In this case, all predictors may be potentially relevant.
- Why did you choose this library (Statsmodels vs scikit-learn)?
    - 
- If you choose Ridge or Lasso, you must try a few λ values and select the best λ  (based on validation logic you clearly explain).

In [38]:
# Prepare features and target variable
features = df[['AT', 'V', 'AP', 'RH']]
target = df['PE']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [39]:
# Helper Functions
def fit_ols_diagnostics(X, y):
    """Fit OLS and return (model, diagnostics dataframe).

    Parameters
    ----------
    X : array-like, shape (n,) or (n, p)
        Feature matrix (without intercept column).
    y : array-like, shape (n,)
        Target vector.
    """
    X = np.asarray(X)
    y = np.asarray(y).reshape(-1)
    if X.ndim == 1:
        X = X.reshape(-1, 1)

    # Add intercept column for statsmodels
    X_sm = sm.add_constant(X)
    model = sm.OLS(y, X_sm).fit()

    infl = OLSInfluence(model)
    diag = pd.DataFrame({
        "y": y,
        "y_hat": model.fittedvalues,
        "residual": model.resid,
        "leverage_hii": infl.hat_matrix_diag,   # diagonal of Hat matrix H
        "cooks_D": infl.cooks_distance[0]
    })
    return model, diag

def plot_line_fit(x, y, model, title=""):
    """Scatter + fitted line for 1D x."""
    x = np.asarray(x).reshape(-1)
    order = np.argsort(x)

    plt.figure(figsize=(7, 4))
    plt.scatter(x, y)
    x_sorted = x[order]

    X_sm = sm.add_constant(x_sorted)
    yhat_sorted = model.predict(X_sm)
    plt.plot(x_sorted, yhat_sorted)

    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Q1: Outlier Detection and Removal using Cook's Distance
model_full, diag_full = fit_ols_diagnostics(X_train, y_train)
print(model_full.summary())

# Define Cook's distance threshold
n = len(X_train)
cooks_threshold = 4 / n

# Identify and sort outliers using Cook's distance
diag_table = diag_full.copy()
diag_table['is_outlier'] = diag_table['cooks_D'] > cooks_threshold
print(f"Cook's distance heuristic threshold 4/n = {cooks_threshold:.3f}")
diag_table.sort_values("cooks_D", ascending=False)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     3109.
Date:                Sat, 24 Jan 2026   Prob (F-statistic):               0.00
Time:                        15:07:21   Log-Likelihood:                -25627.
No. Observations:                6697   AIC:                         5.126e+04
Df Residuals:                    6692   BIC:                         5.130e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         82.5912     23.258      3.551      0.0

,y,y_hat,residual,leverage_hii,cooks_D,is_outlier
2397,370.64803,509.615713,-138.967683,0.037486,1.265702e+00,True
5670,364.32803,559.465487,-195.137457,0.018852,1.207862e+00,True
3919,541.43197,415.727283,125.704687,0.041370,1.152221e+00,True
3248,575.06197,408.386275,166.675695,0.024168,1.142049e+00,True
3477,358.96803,503.918492,-144.950462,0.030533,1.105591e+00,True
...,...,...,...,...,...,...
5449,463.70000,463.686856,0.013144,0.000276,7.718906e-11,False
4540,452.27000,452.258677,0.011323,0.000294,6.102539e-11,False
960,452.13000,452.124884,0.005116,0.000539,2.288271e-11,False
6353,463.46000,463.464109,-0.004109,0.000587,1.607772e-11,False


In [ ]:
idx_remove = list(diag_table[diag_table['is_outlier']].index.tolist())

x_train_idx = X_train.index[idx_remove]
y_train_idx = y_train.index[idx_remove]

X_train_clean = X_train.drop(index=x_train_idx)
y_train_clean = y_train.drop(index=y_train_idx)

model_clean, diag_clean = fit_ols_diagnostics(X_train_clean, y_train_clean)

coef_full = np.asarray(model_full.params).reshape(-1)
coef_clean = np.asarray(model_clean.params).reshape(-1)

print(f"Full-data:   b0 = {coef_full[0]:.4f}, b1 = {coef_full[1]:.4f}")
print(f"Cleaned-data: b0 = {coef_clean[0]:.4f}, b1 = {coef_clean[1]:.4f}")

plot_line_fit(X_train, y_train, model_full, title="Before removal (with influential point)")
plot_line_fit(X_train_clean, y_train_clean, model_clean, title="After removal (refit without influential point)")

Full-data:   b0 = 82.5912, b1 = -1.1106
Cleaned-data: b0 = 453.1256, b1 = -1.9882


KeyError: 0